# Tradução por Máquina: método estatístico e codificador-decodificador

### Trabalho 3 da disciplina Noções de Inteligência Artificial - 2/2024
### Alunos: Felipe Lopes Gibin Duarte (231025207) e Matheus das Neves Fernandes (231013672)

## Introdução

Neste trabalho vamos usar duas técnicas de tradução automática, a primeira puramente
estatística e a segunda usando ML na arquitetura codificador-decodificador. Vamos analisar a eficácia de ambas, destacando aspectos teóricos importantes.

## 1. Base de Dados

### 1.1 Download da base de dados

Usaremos a base Português - Inglês do parlamento europeu, disponível em https://www.statmt.org/europarl/. Encontre o link “parallel corpus Portuguese-English” e extraia o condeúdo do zip. Pela falta de recursos computacionais, o tamanho dos arquivos foi reduzido para 11 mil sentenças cada.

### 1.2 Upload dos arquivos

In [1]:
#Upload dos arquivos extraídos para o google colab
from google.colab import files

uploaded = files.upload()

Saving reduced_europarl_pt.txt to reduced_europarl_pt.txt
Saving reduced_europarl_en.txt to reduced_europarl_en.txt


### 1.3 Processamento dos arquivos

In [2]:
#Limpeza das sentenças
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'-', ' ',text) #trata hifens
    text = re.sub(r'[^\w\s]', '', text) # remove pontuação
    return text.strip()

In [16]:
# Le os arquivos e remove line breaks
with open('/content/reduced_europarl_pt.txt', 'r', encoding='utf-8') as pt_file:
    portuguese_sentences = [clean_text(line.strip()) for line in pt_file.readlines()]

with open('/content/reduced_europarl_en.txt', 'r', encoding='utf-8') as en_file:
    english_sentences = [clean_text(line.strip()) for line in en_file.readlines()]


portuguese_sentences= portuguese_sentences[:10000]
english_sentences= english_sentences[:10000]
assert len(portuguese_sentences) == len(english_sentences)

# Exibir algumas linhas para verificar
print("Frases em português:", portuguese_sentences[:5])
print("Frases em inglês:", english_sentences[:5])

Frases em português: ['reinício da sessão', 'declaro reaberta a sessão do parlamento europeu que tinha sido interrompida na sexta feira 17 de dezembro último e renovo todos os meus votos esperando que tenham tido boas férias', 'como puderam constatar o grande bug do ano 2000 não aconteceu em contrapartida os cidadãos de alguns dos nossos países foram vítimas de catástrofes naturais verdadeiramente terríveis', 'os senhores manifestaram o desejo de se proceder a um debate sobre o assunto nos próximos dias durante este período de sessões', 'entretanto gostaria   como também me foi pedido por um certo número de colegas   que observássemos um minuto de silêncio por todas as vítimas nomeadamente das tempestades nos diferentes países da união europeia que foram afectados']
Frases em inglês: ['resumption of the session', 'i declare resumed the session of the european parliament adjourned on friday 17 december 1999 and i would like once again to wish you a happy new year in the hope that you en

## 2. IBM Model 1

O IBM Model 1 é um modelo complexo de tradução estatística, amplamente utilizado antes da popularização das traduções baseadas em redes neurais. Ele se baseia em probabilidades de palavras individuais e sua correspondência entre os idiomas, ou seja, o modelo estima a melhor tradução palavra por palavra.

### 2.1 Treinando o modelo IBM Model 1

In [21]:
#Instalando o nltk
!pip install nltk

# Tokenização
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')

portuguese_tokens = [word_tokenize(sentence, language='portuguese') for sentence in portuguese_sentences]
english_tokens = [word_tokenize(sentence, language='english') for sentence in english_sentences]

assert len(portuguese_tokens) == len(english_tokens)

# Verificar tokens das primeiras sentenças
print("Português (fonte):", portuguese_tokens[:5])
print("Inglês (alvo):", english_tokens[:5])

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Português (fonte): [['reinício', 'da', 'sessão'], ['declaro', 'reaberta', 'a', 'sessão', 'do', 'parlamento', 'europeu', 'que', 'tinha', 'sido', 'interrompida', 'na', 'sexta', 'feira', '17', 'de', 'dezembro', 'último', 'e', 'renovo', 'todos', 'os', 'meus', 'votos', 'esperando', 'que', 'tenham', 'tido', 'boas', 'férias'], ['como', 'puderam', 'constatar', 'o', 'grande', 'bug', 'do', 'ano', '2000', 'não', 'aconteceu', 'em', 'contrapartida', 'os', 'cidadãos', 'de', 'alguns', 'dos', 'nossos', 'países', 'foram', 'vítimas', 'de', 'catástrofes', 'naturais', 'verdadeiramente', 'terríveis'], ['os', 'senhores', 'manifestaram', 'o', 'desejo', 'de', 'se', 'proceder', 'a', 'um', 'debate', 'sobre', 'o', 'assunto', 'nos', 'próximos', 'dias', 'durante', 'este', 'período', 'de', 'sessões'], ['entretanto', 'gostaria', 'como', 'também', 'me', 'foi', 'pedido', 'por', 'um', 'certo', 'número', 'de', 'colegas', 'que', 'observássemos', 'um', 'minuto', 'de', 'silêncio', 'por', 'todas', 'as', 'vítimas', 'nomeadam

In [5]:
# Criar o corpus alinhado em pares de listas do tipo (target, source) → (Inglês, Português)
from nltk.translate import AlignedSent
parallel_corpus = [AlignedSent(en,pt) for pt, en in zip(portuguese_tokens, english_tokens)]

print(parallel_corpus[:5])

[AlignedSent(['resumption', 'of', 'the', 'session'], ['reinício', 'da', 'sessão'], Alignment([])), AlignedSent(['i', 'declare', 'resumed', 'the', 'session', 'of', 'the', 'european', 'parliament', 'adjourned', 'on', 'friday', '17', 'december', '1999', 'and', 'i', 'would', 'like', 'once', 'again', 'to', 'wish', 'you', 'a', 'happy', 'new', 'year', 'in', 'the', 'hope', 'that', 'you', 'enjoyed', 'a', 'pleasant', 'festive', 'period'], ['declaro', 'reaberta', 'a', 'sessão', 'do', 'parlamento', 'europeu', 'que', 'tinha', 'sido', 'interrompida', 'na', 'sexta', 'feira', '17', 'de', 'dezembro', 'último', 'e', 'renovo', 'todos', 'os', 'meus', 'votos', 'esperando', 'que', 'tenham', 'tido', 'boas', 'férias'], Alignment([])), AlignedSent(['although', 'as', 'you', 'will', 'have', 'seen', 'the', 'dreaded', 'millennium', 'bug', 'failed', 'to', 'materialise', 'still', 'the', 'people', 'in', 'a', 'number', 'of', 'countries', 'suffered', 'a', 'series', 'of', 'natural', 'disasters', 'that', 'truly', 'were',

In [6]:
# Treinamento do Modelo (direção correta)
from nltk.translate import IBMModel1
model = IBMModel1(parallel_corpus, 20)  # 20 iterações

print("Modelo treinado com sucesso!")

Modelo treinado com sucesso!


### 2.2 Avaliando a qualidade da tradução

In [19]:
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
import numpy as np

# Função de suavização para o BLEU score
smoothing = SmoothingFunction().method1

# 1. Criar o dicionário de tradução
translation_dict = {}
for en_word in model.translation_table:
    for pt_word in model.translation_table[en_word]:
        prob = model.translation_table[en_word][pt_word]
        if pt_word not in translation_dict or prob > translation_dict[pt_word][1]:
            translation_dict[pt_word] = (en_word, prob)

# Exibir uma parte do dicionário de tradução
print("Dicionário de tradução (exemplo):", list(translation_dict.items())[:5])

# 2. Função de Tradução
def translate(source_tokens):
    return [translation_dict.get(tok, ('UNK', 0))[0] for tok in source_tokens]

# Vamos usar as sentenças em inglês como referências e as sentenças traduzidas como hipóteses
references = [[eng] for eng in english_tokens]
hypotheses = [translate(pt) for pt in portuguese_tokens]

#debugging
print(len(references))
print(len(hypotheses))


# 5. Calcular o BLEU score médio
bleu_score = corpus_bleu(references, hypotheses, weights=(0.5, 0.5), smoothing_function=smoothing)
print(f"BLEU Score médio: {bleu_score:.4f}")


Dicionário de tradução (exemplo): [(None, ('the', 0.36681815178192073)), ('poderá', ('can', 0.3143122775494104)), ('a', ('the', 0.2625850446893964)), ('comissão', ('commission', 0.9955597345022094)), ('assegurar', ('ensure', 0.7091438485543141))]
10716
11592


AssertionError: The number of hypotheses and their reference(s) should be the same 

In [8]:
# Exemplo: Ver traduções para a palavra "sim"
word = "sim"
if word in model.translation_table:
    translations = model.translation_table[word]
    sorted_translations = sorted(translations.items(), key=lambda x: -x[1])
    print(f"Traduções para '{word}': {sorted_translations[:5]}")  # Top 5
else:
    print(f"'{word}' não encontrada no modelo.")

Traduções para 'sim': [('yes', 0.6672324401779459), ('deflation', 0.19811735634075164), ('definite', 0.16528434773779155), ('philosophical', 0.16401117047168076), ('justification', 0.1572788962025973)]


In [14]:
#tradução de uma frase
sentence1=portuguese_tokens[0]
sentence2=portuguese_tokens[8]

print(sentence1)
print(translate_sentence(model,sentence1))
print("-" * 50)

print(sentence2)
print(translate_sentence(model,sentence2))
print("-" * 50)

['reinício', 'da', 'sessão']
['resumption', 's', 'session']
--------------------------------------------------
['certamente', 'que', 'já', 'tomou', 'conhecimento', 'pelas', 'notícias', 'transmitidas', 'na', 'imprensa', 'e', 'na', 'televisão', 'dos', 'diversos', 'atentados', 'à', 'bomba', 'e', 'assassínios', 'perpetrados', 'no', 'sri', 'lanka']
['certainly', 'which', 'already', 'seizing', 'knowledge', 'determined', 'news', 'relayed', 'afsj', 'press', 'and', 'afsj', 'television', 'vessels', 'various', 'explosions', 'starts', 'bomb', 'and', 'murders', 'explosions', 'within', 'lanka', 'lanka']
--------------------------------------------------


### 2.3 Considerações sobre o BLEU score

O BLEU score (Bilingual Evaluation Understudy) é uma métrica usada para avaliar a qualidade de traduções automáticas, comparando a tradução gerada pelo modelo com uma ou mais traduções de referência. No caso do IBM model 1, o resultado encontrado foi desapontador devido à abordagem de tradução palavra a palavra, que é ineficiente. Alem disso, a limitação em tamanho da base de dados devido à restrições de capacidade computacional foi outro fator relevante para o score médio ter sido tão baixo (0.0294).  

## 3. Codificador-Decodificador

### 3.1 Upload e processamento dos dados

In [ ]:
# Exibir algumas linhas para verificar
print("Frases em português:", portuguese_sentences[:5])
print("Frases em inglês:", english_sentences[:5])

Frases em português: ['Reinício da sessão', 'Declaro reaberta a sessão do Parlamento Europeu, que tinha sido interrompida na sexta-feira, 17 de Dezembro último, e renovo todos os meus votos, esperando que tenham tido boas férias.', 'Como puderam constatar, o grande "bug do ano 2000" não aconteceu. Em contrapartida, os cidadãos de alguns dos nossos países foram vítimas de catástrofes naturais verdadeiramente terríveis.', 'Os senhores manifestaram o desejo de se proceder a um debate sobre o assunto nos próximos dias, durante este período de sessões.', 'Entretanto, gostaria - como também me foi pedido por um certo número de colegas - que observássemos um minuto de silêncio por todas as vítimas, nomeadamente das tempestades, nos diferentes países da União Europeia que foram afectados.']
Frases em inglês: ['Resumption of the session', 'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in th

### 3.2 Importações e configurações


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Input
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Configurações
max_vocab_size = 25000  # Tamanho máximo do vocabulário
max_seq_length = 30     # Tamanho máximo da sequência
embedding_dim = 256     # Dimensão do embedding
latent_dim = 512        # Dimensão do vetor latente (de contexto) do RNN
batch_size = 64
epochs = 20

### 3.3 Tokenização e padding

A tokenização converte palavras em números, onde cada número representa o índice de uma palavra no vocabulário. Palavras associadas a índices de número mais baixo são mais frequentes no texto. O token "start" e "end" indicam onde a tradução começa e onde termina. O padding garante que todas as sentenças tenham o mesmo comprimento, se for uma frase mais curta que max__seq_length, serão adicionados zeros ao final das frases.

In [ ]:
#Adicionamos tokens de inicio e fim em ingles
english_sentences = ["<start> " + sentence.strip() + " <end>" for sentence in english_sentences]
print(english_sentences[1])

<start> I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period. <end>


In [ ]:
#tokenização
tokenizer_pt = Tokenizer(num_words=max_vocab_size)
tokenizer_pt.fit_on_texts(portuguese_sentences)

tokenizer_en = Tokenizer(num_words=max_vocab_size)
tokenizer_en.fit_on_texts(english_sentences)

In [ ]:
# Sequências numericas das sentenças em portugues e ingles
input_sequences = tokenizer_pt.texts_to_sequences(portuguese_sentences)
target_sequences = tokenizer_en.texts_to_sequences(english_sentences)

In [ ]:
#Adiciona padding ao final das sentenças, de forma a padronizar o tamanho delas
input_sequences = pad_sequences(input_sequences, maxlen=max_seq_length, padding="post")
target_sequences = pad_sequences(target_sequences, maxlen=max_seq_length, padding="post")


In [ ]:
#Criação dos arrays de entrada e saida
encoder_input_data = np.array(input_sequences)
decoder_input_data = pad_sequences(target_sequences[:, :-1], maxlen=max_seq_length, padding="post")  # Sem o token <end>
decoder_target_data = pad_sequences(target_sequences[:, 1:], maxlen=max_seq_length, padding="post")  # Sem o token <start>

# Corrigindo erro de tamanho dos arrays
min_samples = min(len(encoder_input_data), len(decoder_input_data), len(decoder_target_data))

encoder_input_data = encoder_input_data[:min_samples]
decoder_input_data = decoder_input_data[:min_samples]
decoder_target_data = decoder_target_data[:min_samples]

# Verifique novamente os tamanhos
print(f"encoder_input_data shape: {encoder_input_data.shape}")
print(f"decoder_input_data shape: {decoder_input_data.shape}")
print(f"decoder_target_data shape: {decoder_target_data.shape}")

### 3.4 Estrutura do modelo encoder-decoder

O encoder processa a sequência de entrada (sentenças em português) e gera um vetor de contexto que contém a representação da sequência. O vetor de contexto é então passado para o decodificador. Neste caso, encoder_outputs pode ser ignorado, só estamos interessados no vetor state_h.

In [ ]:
# Definindo o encoder

# A entrada é uma sequência de índices (tokens) representando palavras em português
encoder_inputs = Input(shape=(max_seq_length,))

#Embedding para representar cada palavra como um vetor
encoder_embedding = Embedding(input_dim=max_vocab_size, output_dim=embedding_dim)(encoder_inputs)

#RNN que vai gerar o vetor de estado
encoder_rnn = SimpleRNN(latent_dim, return_state=True)
encoder_outputs, state_h = encoder_rnn(encoder_embedding)

O decoder usa o estado latente (state_h) gerado pelo encoder para produzir a tradução. Ele precisa ser alimentado com o token "start" no início, gerando uma palavra por vez até encontrar o token "end".

In [ ]:
# Definindo o decoder

#Sequencia de tokens em ingles, sem o <end>
decoder_inputs = Input(shape=(max_seq_length,))

#Embedding dos tokens em ingles
decoder_embedding = Embedding(input_dim=max_vocab_size, output_dim=embedding_dim)(decoder_inputs)

#RNN do decoder
#decoder_outputs: Saída para cada passo temporal da RNN no decoder.
#A RNN usa o estado final do encoder (state_h) como estado inicial.
decoder_rnn = SimpleRNN(latent_dim, return_sequences=True)
decoder_outputs = decoder_rnn(decoder_embedding, initial_state=state_h)

#Camada densa para prever a próxima palavra
# A softmax prevê a próxima palavra em inglês como uma probabilidade sobre o vocabulário
decoder_dense = Dense(max_vocab_size, activation="softmax",kernel_regularizer=regularizers.l2(0.01))
decoder_outputs = decoder_dense(decoder_outputs)

Agora, combinamos o encoder e o decoder para montar o modelo completo que mapeará as frases em ingles para as suas traduções em inglês.

In [ ]:
#O modelo recebe como entrada encoder_inputs e decoder_inputs e tem como saida decoder_outputs (frases em ingles traduzidas)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

### 3.5 Treinamento

In [ ]:
#Compilação do modelo completo

model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

In [ ]:
#Treinamento do modelo

#Early stopping
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True)

model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split = 0.2,  # 20% para validação.
    callbacks=[es]
)

Epoch 1/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 11s 83ms/step - accuracy: 0.5256 - loss: 2.4318 - val_accuracy: 0.4339 - val_loss: 4.0142
Epoch 2/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 20s 79ms/step - accuracy: 0.5405 - loss: 2.3184 - val_accuracy: 0.4346 - val_loss: 4.0479
Epoch 3/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 20s 77ms/step - accuracy: 0.5548 - loss: 2.2103 - val_accuracy: 0.4343 - val_loss: 4.0797
Epoch 4/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 20s 73ms/step - accuracy: 0.5728 - loss: 2.0962 - val_accuracy: 0.4317 - val_loss: 4.1222
Epoch 5/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 11s 80ms/step - accuracy: 0.5890 - loss: 1.9943 - val_accuracy: 0.4285 - val_loss: 4.1543
Epoch 6/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 20s 73ms/step - accuracy: 0.6007 - loss: 1.9252 - val_accuracy: 0.4323 - val_loss: 4.1938
Epoch 7/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 10s 74ms/step - accuracy: 0.6193 - loss: 1.8216 - val_accuracy: 0.4275 - val_loss: 4.2353


NameError: name 'accuracy' is not defined

### 3.6 Tradução de frases simples

A arquitetura que criamos é para o treinamento. Para usar o modelo treinado na prática, precisamos configurar o encoder e decoder para inferência. Na inferência (tradução), o processo é diferente porque não temos a frase-alvo completa (em inglês). A tradução precisa ser gerada uma palavra por vez, de forma iterativa.

In [ ]:
#Encoder para inferencia
encoder_model = Model(encoder_inputs, encoder_states)

NameError: name 'encoder_states' is not defined

In [ ]:
#Decoder para inferencia
#Recebe os estados latentes do encoder como entrada e gera palavras passo a passo

# Entradas para os estados iniciais do decoder
decoder_state_input = Input(shape=(latent_dim,))

# Entrada da palavra atual
decoder_single_input = Input(shape=(1,))  # Entrada com uma palavra por vez

# Embedding para a palavra atual
decoder_embedded = decoder_embedding(decoder_single_input)

# RNN usando o estado anterior
decoder_output, decoder_state = decoder_rnn(decoder_embedded, initial_state=decoder_state_input)

# Camada densa para prever a próxima palavra
decoder_prediction = decoder_dense(decoder_output)

# Modelo do decoder para inferência
decoder_model = Model(
    [decoder_single_input, decoder_state_input],  # Entradas: palavra + estado anterior
    [decoder_prediction, decoder_state]          # Saídas: previsão + estado atual
)

Criaremos uma função de tradução que passa uma frase em portugûes e retorna a frase traduzida para o inglês. Após isso, traduziremos uma frase simples

In [ ]:
def translate_sentence(input_sequence, encoder_model, decoder_model, tokenizer_en, max_seq_length):
    # Obtenha o estado latente do encoder
    encoder_states = encoder_model.predict(input_sequence)

    # Comece a tradução com o token <start>
    target_sequence = np.zeros((1, 1))  # Tamanho (1 frase, 1 palavra)
    target_sequence[0, 0] = tokenizer_en.word_index["<start>"]

    decoded_sentence = []

    # Iterar para gerar palavras até o token <end> ou o limite de comprimento
    for _ in range(max_seq_length):
        # Previsão da próxima palavra
        output_tokens, h = decoder_model.predict([target_sequence] + [encoder_states])

        # Palavra com maior probabilidade
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = tokenizer_en.index_word[sampled_token_index]

        # Adiciona a palavra à sentença decodificada
        if sampled_word == "<end>":
            break
        decoded_sentence.append(sampled_word)

        # Atualize a sequência de entrada para o próximo passo
        target_sequence = np.zeros((1, 1))
        target_sequence[0, 0] = sampled_token_index

        # Atualize os estados do decoder
        encoder_states = h

    return " ".join(decoded_sentence)


In [ ]:
# Teste de uma frase simples

### 3.7 Avaliação

Avaliaremos o modelo usando o BLEU score, novamente. Espera-se que a tradução seja mais eficiente pelo fato de RNNs serem capazes de capturar o contexto das frases, gerando traduções mais naturais e eficientes.